In [1]:
import pandas as pd
import numpy as np
import folium
from folium.plugins import TimestampedGeoJson

In [2]:
# importing the file needed for the analysis
df = pd.read_csv("../data/raw/sales_enriched2.csv")

In [3]:
df.head()

,sale_date,product,flavour,customization,quantity_value,quantity_units,price,mode_of_payment,toppers,delivery,...,latitude,longitude,product_raw,flavour_raw,year_,month_,day_,dow_,week_,quarter_
0,17-10-2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2022,10,17,1,42,4
1,01-07-2022,cake,mango,NaN,500.0,gram,500.0,acc,NaN,NaN,...,NaN,NaN,cake,mango,2022,7,1,5,26,3
2,03-06-2021,cake,mango,NaN,500.0,gram,450.0,acc,NaN,no,...,NaN,NaN,cake,mango,2021,6,3,4,22,2
3,15-04-2022,cake,mango,NaN,1000.0,gram,900.0,acc,NaN,NaN,...,NaN,NaN,cake,mango,2022,4,15,5,15,2
4,09-11-2022,NaN,brownie,NaN,15.0,pcs,975.0,acc,NaN,NaN,...,NaN,NaN,NaN,brownies,2022,11,9,3,45,4


In [4]:
df = df[['sale_date', 'latitude', 'longitude']].sort_values('sale_date').dropna()

In [5]:
df['sale_date'] = pd.to_datetime(df['sale_date'], format='%d-%m-%Y')

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 762 entries, 332 to 362
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   sale_date  762 non-null    datetime64[ns]
 1   latitude   762 non-null    float64       
 2   longitude  762 non-null    float64       
dtypes: datetime64[ns](1), float64(2)
memory usage: 23.8 KB


In [7]:
df.head()

,sale_date,latitude,longitude
332,2021-01-01,11.07290,77.10083
59,2021-01-01,11.05851,77.07965
453,2021-01-01,11.00992,77.07994
610,2022-01-01,11.02622,77.01611
72,2021-02-01,11.04621,77.01415


# Delivery locations animated

In [8]:
# Create a Folium map centered at an initial location
m = folium.Map(location=[11.063003, 77.083743], zoom_start=12)

# Create a list to store the GeoJSON features
features = []

# Loop through your dataframe and create GeoJSON features
for index, row in df.iterrows():
    feature = {
        'type': 'Feature',
        'geometry': {
            'type': 'Point',
            'coordinates': [row['longitude'], row['latitude']]
        },
        'properties': {
            'time': row['sale_date'].strftime('%Y-%m-%d'),  # Format date as string
            'popup': f"Date: {row['sale_date'].strftime('%Y-%m-%d')}<br>Latitude: {row['latitude']}<br>Longitude: {row['longitude']}"
        }
    }
    features.append(feature)

# Create a GeoJSON object with the list of features
geojson = {
    'type': 'FeatureCollection',
    'features': features
}

# Add TimestampedGeoJson plugin to the map
TimestampedGeoJson(geojson,
                   period='P10D',  # Animation interval (1 day in this case)
                   add_last_point=True,  # Add the last point after animation
                   auto_play=True).add_to(m)

# Save the map as an HTML file
m
